# Moving from Bayesflow 1.0 to 2.0

Current users of bayesflow will notice that with the update to 2.0 many things have changed or been moved. This short guide aims to clarify what has been changed as well as additonal functionalities that have been added. This guide follows a similar structure to the Quickstart notebook, without the mathematical explaination in order to demonstrate the differences in workflow. However, for a more detailed explaination of any of the features, users should read any of the other example notebooks. 

## Major Changes 

One of the major changes from _bayesflow 1.0_ to _bayeflow 2.0_ is that entire package has been reformatted in line with keras standards. This was done to allow users to choose their prefered backend for machine learning models. Rather than only being compatible with tensorflow, users can now choose to fit their models with either `TensorFlow`, `JAX` or `Pytorch`. 


In [ ]:
import numpy as np

# ensure the backend is set
import os
if "KERAS_BACKEND" not in os.environ:
    # set this to "torch", "tensorflow", or "jax"
    os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import bayesflow as bf

This version of bayeflow also relies much more heavily on dictionaries. Nearly every object, and function will expect a dictionary, so any parameter or data should be returned as a dictionary. 

## Example Workflow 

### 1. Priors and Likelihood Model

Previously users would define a prior function, which would then be used by a `Prior` object to sample prior values. The likelihood would then also be specified via function and used by a `Simulator` wrapper to produce observations for a given prior. These were then combined in the `GenerativeModel`, however this has been changed, we no longer use the `Prior`, `Simulator` or `GenerativeModel` objects. Instead `GenerativeModel` has been renamed to `simulator` which is a  single function that glues the prior and likelihood together.  

In [ ]:
def theta_prior():
    theta = np.random.normal(size=4)
    # previously: 
    # return theta 
    return dict(theta=theta) # notice we return a dictionary
    

def likelihood_model(theta, n_obs):
    x = np.random.normal(loc=theta, size=(n_obs, theta.shape[0]))
    return dict(x=x)


Previously the prior and likelihood were defined as

In [ ]:
# Do Not Run
prior = bf.simulation.Prior(prior_fun=theta_prior)
simulator = bf.simulation.Simulator(simulator_fun=likelihood_model)

Within the new framework we also a define a meta function which allows us to dynamically set the batch size. 

In [ ]:
def meta(batch_size):
    return dict(n_obs=50)

simulator = bf.make_simulator([theta_prior, likelihood_model], meta_fn=meta)

### 2. Adapter and Data Configuration

In _bayesflow2.0_ we now need to specify the data configuration. For example we should specify which variables are `summary_variables` meaning observations that will be summarized in the summary network, the `inference_variables` meaning the prior draws on which we're interested in training the posterior network and the `inference_conditions` which specify our number of observations. Previously these things were inferred from the type of network used, but now they should be defined explictly with  the `adapter`. This allows users to ???  

In [ ]:
adapter = (
    bf.adapters.Adapter()
    .to_array()
    .broadcast("n_obs")
    .convert_dtype(from_dtype="float64", to_dtype="float32")
    .standardize(exclude=["n_obs"])
    .rename("x", "summary_variables")
    .rename("theta", "inference_variables")
    .rename("n_obs", "inference_conditions")
)

In addition the adapter now has built in functions to transform data such as standardization or one-hot encoding. For a full list of the adapter transforms, please see the documentation. 

### 3. Summary Network and Inference Network

### 4. Approximator (Amortizer Posterior)